In [44]:
import sklearn.metrics
from sklearn.ensemble import RandomForestRegressor
from fastai.imports import *
from fastai.structured import *
from sklearn.model_selection import train_test_split

In [45]:
main_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/train.csv')
meal_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/meal_info.csv')
center_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/fulfilment_center_info.csv')

df_raw = pd.merge(main_raw, meal_raw,how='outer', on='meal_id')
df_raw = pd.merge(df_raw, center_raw, how='outer', on='center_id'); df_raw.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine,city_code,region_code,center_type,op_area
0,1379560,1,55,1885,136.83,152.29,0,0,177,Beverages,Thai,647,56,TYPE_C,2.0
1,1018704,2,55,1885,135.83,152.29,0,0,323,Beverages,Thai,647,56,TYPE_C,2.0
2,1196273,3,55,1885,132.92,133.92,0,0,96,Beverages,Thai,647,56,TYPE_C,2.0
3,1116527,4,55,1885,135.86,134.86,0,0,163,Beverages,Thai,647,56,TYPE_C,2.0
4,1343872,5,55,1885,146.50,147.50,0,0,215,Beverages,Thai,647,56,TYPE_C,2.0


In [46]:
df_raw['discount'] = df_raw.base_price - df_raw.checkout_price
df_raw['discount_ratio'] = df_raw['discount'] / df_raw['base_price']
df_w = df_raw
df_w['month'] = df_raw.week / 4
df_w['month'] = df_w['month'].astype('int')
df_w['week_mod52'] = df_raw['week'] % 52
df_w['week_mod4'] = df_raw['week'] % 4
df_w.drop('id', axis=1, inplace=True)
train_cats(df_w)
df_trn, y_trn, nas = proc_df(df_w, 'num_orders')

In [47]:
df_trn.head()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,category,cuisine,city_code,region_code,center_type,op_area,discount,discount_ratio,month,week_mod52,week_mod4
0,1,55,1885,136.83,152.29,0,0,1,4,647,56,3,2.0,15.46,0.101517,0,1,1
1,2,55,1885,135.83,152.29,0,0,1,4,647,56,3,2.0,16.46,0.108083,0,2,2
2,3,55,1885,132.92,133.92,0,0,1,4,647,56,3,2.0,1.00,0.007467,0,3,3
3,4,55,1885,135.86,134.86,0,0,1,4,647,56,3,2.0,-1.00,-0.007415,1,4,0
4,5,55,1885,146.50,147.50,0,0,1,4,647,56,3,2.0,1.00,0.006780,1,5,1


In [48]:
y_trn

array([177, 323,  96, ...,  40,  53,  27])

In [49]:
m = RandomForestRegressor(n_estimators=100, n_jobs=-1, max_features=0.3, min_samples_leaf=2, oob_score=False)
m.fit(df_trn, y_trn)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.3, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [50]:
_, x_test, _, y_test = train_test_split(df_trn, y_trn, test_size=0.99)

In [51]:
len(y_test)

451983

In [52]:
metrics.mean_squared_log_error(y_test, m.predict(x_test))*100

9.90321069873133

In [53]:
test_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/test_QoiMO9B.csv')
meal_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/meal_info.csv')
center_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/fulfilment_center_info.csv')

test_raw = pd.merge(test_raw, meal_raw,how='outer', on='meal_id')
test_raw = pd.merge(test_raw, center_raw, how='outer', on='center_id')

test_raw['discount'] = test_raw.base_price - test_raw.checkout_price
test_raw['discount_ratio'] = test_raw['discount'] / test_raw['base_price']
test_w = test_raw
test_w['month'] = test_raw.week / 4
test_w['month'] = test_w['month'].astype('int')
test_w['week_mod52'] = test_raw['week'] % 52
test_w['week_mod4'] = test_raw['week'] % 4
test_w.drop('id', axis=1, inplace=True)
train_cats(test_w)
test_w, _, _ = proc_df(test_w)

In [54]:
preds = m.predict(test_w)

In [56]:
res = pd.DataFrame(preds)

test_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/test_QoiMO9B.csv')

res['id'] = test_raw.id

res['num_orders'] = preds

res.head()

res.columns

res.drop(0, axis=1, inplace=True)

res.head()

res.to_csv('/home/orange/Desktop/result.csv')

In [57]:
fi = rf_feat_importance(m, df_trn); fi[:25]

,cols,imp
3,checkout_price,0.127970
7,category,0.122950
4,base_price,0.087421
12,op_area,0.085263
2,meal_id,0.080546
6,homepage_featured,0.064189
14,discount_ratio,0.063740
8,cuisine,0.056696
5,emailer_for_promotion,0.054071
1,center_id,0.048467
